In [1]:
import pandas as pd 
import sqlite3 
import numpy as np 
import datetime
import sys 
import os 
from ast import literal_eval
%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append('/home/malcolm/WorkTransit/scripts/WorkTransitLib/')

In [3]:
import parseRawResponse as parser

In [4]:
transit_parser = parser.RunParser('transit', 'parsed_transit')
transit_parser.execute()

Tables:  ['Raw_Responses', 'parsed_driving', 'parsed_transit']
Latest date for parsed_transit is 2021-09-25 21:14
Raw Response df has  0  rows
parsed_transit  was updated with  0  rows


/home/malcolm/main/lib/python3.6/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [5]:
driving_parser = parser.RunParser('driving', 'parsed_driving')
driving_parser.execute()

Tables:  ['Raw_Responses', 'parsed_driving', 'parsed_transit']
Latest date for parsed_driving is 2021-09-25 22:32
Raw Response df has  0  rows
parsed_driving  was updated with  0  rows


In [6]:
raise ValueError()

ValueError: 

## MANUAL run 0925

In [7]:
driving_parser.raw_df = driving_parser.get_raw_responses_df(driving_parser.latest_date, '00:00')
driving_parser.parsed_df = driving_parser.parse_raw_df(driving_parser.raw_df)
driving_parser.full_df = driving_parser.merge(driving_parser.raw_df, driving_parser.parsed_df)
driving_parser.update_table(driving_parser.full_df)


Raw Response df has  382  rows
parsed_driving  was updated with  382  rows


In [8]:
transit_parser.raw_df = transit_parser.get_raw_responses_df(transit_parser.latest_date, '00:00')
transit_parser.parsed_df = transit_parser.parse_raw_df(transit_parser.raw_df)
transit_parser.full_df = transit_parser.merge(transit_parser.raw_df, transit_parser.parsed_df)
transit_parser.update_table(transit_parser.full_df)


Raw Response df has  116  rows


IndexError: list index out of range

## Graveyard

In [ ]:
con = sqlite3.connect('/home/malcolm/WorkTransit/data/transit.db')
cursor = con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\' ORDER BY name')
tables = cursor.fetchall()
tables = [x[0] for x in tables]
tables

In [ ]:
latest_transit_date, latest_transit_time = cursor.execute("select max(date) as date, max(time) as time from parsed_transit").fetchall()[0]
# latest_driving_date = cursor.execute("select max(date) as date, max(time) as time from parsed_driving").fetchall()[0][0]
# print("Latest Transit Date: ", latest_transit_date, " Latest Driving Date: ", latest_driving_date)

latest_transit_date, latest_transit_time


In [ ]:
    def get_raw_responses_df(self, latest_date, latest_time):
        raw_df = pd.read_sql(f"""
            select * from raw_responses 
            where Date >= '{latest_date}'
            and Time >= '{latest_time}'
            and mode = '{self.mode}'
            """, self.con)
        print("Raw Response df has ", raw_df.shape[0], " rows")
        return(raw_df)

In [ ]:
latest_transit_time = '07:03'

In [ ]:
transit_df_raw = pd.read_sql(f"""
select * from raw_responses 
where Date > '{latest_transit_date}'
and mode = 'transit'
and Time >= '{latest_transit_time}'
""", con)
transit_df_raw

In [ ]:
driving_df_raw = pd.read_sql(f"""
select * from raw_responses 
where Date > '{latest_driving_date}'
and mode = 'driving'
""", con)
driving_df_raw

In [ ]:
transit_df_parsed = transit_df_raw['Raw Response']\
    .apply(lambda x: parser.parse_transit_details(literal_eval(x)[0]))
transit_df_parsed

In [ ]:
driving_df_parsed = driving_df_raw['Raw Response']\
    .apply(lambda x: parser.parse_driving_details(literal_eval(x)[0]))
driving_df_parsed

In [ ]:
driving_full = pd.merge(driving_df_parsed, driving_df_raw
         , left_index=True, right_index=True)
transit_full = pd.merge(transit_df_parsed, transit_df_raw
                       , left_index=True, right_index=True)


In [ ]:
transit_full.head()

In [ ]:
transit_full.to_sql('parsed_transit', con, index=False, if_exists='append')
con.commit()

In [ ]:
driving_full.to_sql('parsed_driving', con, index=False, if_exists='append')
con.commit()

In [ ]:
con.close()